In [ ]:
# install dependencies
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_adaptive.git
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install -q pyterrier_pisa

In [ ]:
# imports
import pyterrier as pt
pt.init()
from pyterrier.measures import *
from pyterrier_adaptive import GAR, CorpusGraph
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex

In [ ]:
# Create required components
dataset = pt.get_dataset('irds:msmarco-passage')
retriever = PisaIndex.from_dataset('msmarco_passage').bm25()
scorer = pt.text.get_text(dataset, 'text') >> MonoT5ReRanker(verbose=False, batch_size=16)
graph = CorpusGraph.from_dataset('msmarco_passage', 'corpusgraph_bm25_k16').to_limit_k(8)

In [ ]:
# A simple example
pipeline = retriever >> GAR(scorer, graph) >> pt.text.get_text(dataset, 'text')

pipeline.search('clustering hypothesis information retrieval')

In [ ]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
pt.Experiment(
    [retriever, retriever >> scorer, retriever >> GAR(scorer, graph)],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@1000],
    names=['bm25', 'bm25 >> monot5', 'bm25 >> GAR(monot5)']
)